In [87]:
import pandas as pd

In [88]:
df = pd.read_csv('data/serieA_matches.csv')

In [89]:
cols_types = df.dtypes
cols_types[cols_types == object]

team       object
result     object
stadium    object
state      object
UF         object
date       object
time       object
dtype: object

In [90]:
#

In [91]:
df.head(2)

,fouls,corners,crosses,touches,tackles,interceptions,aerials_won,clearances,offsides,goal_kicks,...,red_cards_mean_attack,fouls_commited_mean_attack,fouls_drawn_mean_attack,offsides_mean_attack,crosses_mean_attack,tackles_on_mean_attack,interceptions_mean_attack,own_goals_mean_attack,penalty_kicks_won_mean_attack,penalty_kicks_conceded_mean_attack
0,12.0,3.0,16.0,455,7.0,9.0,8.0,6.0,1.0,6.0,...,0.0,1.0,0.0,0.00,0.5,0.00,0.00,0.0,NaN,NaN
1,7.0,4.0,8.0,590,8.0,5.0,9.0,13.0,1.0,8.0,...,0.0,0.0,1.5,0.25,0.5,0.75,0.25,0.0,NaN,NaN


In [92]:
df['date'] = pd.to_datetime(df['date'])
df['time'] = df['time'].apply(lambda time: time.split(':')[0]).astype(int)

In [93]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter
df['day'] = df['date'].dt.day

In [94]:
# score
# result

df.drop(columns=['score'], inplace=True)

In [95]:
result_int = {
    'L': 0,
    'D': 1,
    'W': 3,
}
df['result'] = df['result'].apply(lambda result: result_int[result]) 

In [96]:
from sklearn.feature_selection import RFE

In [97]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [98]:
df.drop(columns=[
    "date",
    'team',
    "stadium",
    "state",
    "UF"
], inplace=True)

In [99]:
X = df.drop(columns=['result'])
Y = df['result']

In [100]:
# model = XGBClassifier()

# rfe = RFE(model, 25)
# fit = rfe.fit(X, Y)
# print("Num Features: %d" % fit.n_features_)
# print("Selected Features: %s" % fit.support_)
# print("Feature Ranking: %s" % fit.ranking_)

In [101]:
X.fillna(0, inplace=True)

In [102]:
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import f_classif, chi2

# test = SelectKBest(score_func=f_classif, k=5)
# x_new = test.fit_transform(X, Y)

In [103]:
# X = x_new

In [104]:
import numpy as np
# cols = np.array(np.array(X.columns.tolist()))[list(fit.support_)]

In [105]:
# X = X[cols]

In [106]:
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=7)

In [110]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif, chi2
    
n_cols_list = [i for i in range(1, 111)]
best_n = 1
best_acc = 10
for n_cols in n_cols_list:
# n_cols = 5

    model = XGBClassifier()

#     test = SelectKBest(score_func=f_classif, k=n_cols)
#     x_new = test.fit_transform(X, Y)
    rfe = RFE(model, n_cols)
    fit = rfe.fit(X, Y)

    cols = np.array(np.array(X.columns.tolist()))[list(fit.support_)]
    x_new = X[cols]
    
    X_train, X_test, y_train, y_test = train_test_split(x_new, Y, test_size=0.2, random_state=7)
    model.fit(X_train, y_train)

    y_pred = list(model.predict(X_test))
    predictions = list([round(value) for value in y_pred])

    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    if accuracy * 100.0 > best_acc:
        best_acc = accuracy * 100.0
        best_n = n_cols

print("Accuracy with {} columns: {:.2f}%".format(best_n, best_acc))

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_features_to_select=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_features_to_select=2 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_features_to_select=3 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_features_to_select=4 as keyword args. From version 0.25 passing these as positional argu

Accuracy with 5 columns: 66.00%
